# Tensorflow

## Conceptos básicos

1. Se crea una sesión
2. Se define un grafo computacional
3. Se alimenta la información al grafo para recibir una salida

![Grafo](grafo.png)

In [1]:
import tensorflow as tf

# Sólo informe sobre errores

tf.logging.set_verbosity(tf.logging.ERROR)

# Encapsula el ambiente para ejecutar el grafo

session = tf.InteractiveSession() # interactivo
# session = tf.Session() # Implementación


In [2]:
# Construye el grafo

a = tf.placeholder(tf.float32)
b = tf.placeholder(tf.float32)

k = tf.constant(1.0)

c = tf.add(a,b)
d = tf.add(b,k)

e = tf.multiply(c,d)

In [3]:
session.run(e, feed_dict={a:2, b:1})

6.0

In [4]:
session.run(c, feed_dict={a:2, b:1})

3.0

In [5]:
session.run([e,c], feed_dict={a:2, b:1})

[6.0, 3.0]

In [6]:
session.run([e,c], feed_dict={a:[1,2,0], b:[3,1,2]})

[array([16.,  6.,  6.], dtype=float32), array([4., 3., 2.], dtype=float32)]

In [7]:
session.run([e,c], feed_dict={a:[[1,2,0],[0,2,1]], b:[[3,1,2],[2,1,3]]})

[array([[16.,  6.,  6.],
        [ 6.,  6., 16.]], dtype=float32), array([[4., 3., 2.],
        [2., 3., 4.]], dtype=float32)]

In [8]:
from pandas import read_csv
from pandas import get_dummies

df = read_csv("abandono.csv")

values = df.values
# ensure all data is float
values = values.astype('float32')

observaciones = values[:,1:8]
etiquetas = values[:,8].reshape(-1,1)

observaciones = session.run(tf.nn.l2_normalize(observaciones,1))

In [9]:
# construyendo el grafo de un perceptrón

n_nodos_entrada = observaciones.shape[1] 
n_nodos_salida = etiquetas.shape[1]

x = tf.placeholder(tf.float32, (None, n_nodos_entrada))
y = tf.placeholder(tf.float32, (None, n_nodos_salida))

W = tf.Variable(tf.random_normal((n_nodos_entrada, n_nodos_salida)))
b = tf.Variable(tf.zeros(n_nodos_salida))

salida = tf.nn.sigmoid(tf.matmul(x,W) + b)

costo = tf.reduce_mean(
    tf.nn.sigmoid_cross_entropy_with_logits(
        logits=salida, labels=y
    )
)

tasa_de_aprendizaje = 0.1

#optimizer = tf.train.GradientDescentOptimizer(tasa_de_aprendizaje).minimize(costo)

optimizer = tf.train.AdamOptimizer(tasa_de_aprendizaje).minimize(costo)

In [10]:

tf.global_variables_initializer().run()

for _ in range(2000):
    _,w,c=session.run([optimizer, W, costo],feed_dict={x:observaciones,y:etiquetas})


In [11]:
#calculamos que tan certero es nuestro modelo

y_ = tf.placeholder(tf.float32, (None, n_nodos_salida))

predicciones_correctas = tf.equal(salida>.5, y_>.5)

certeza = tf.reduce_mean(tf.cast(predicciones_correctas, tf.float32))

print(session.run(certeza, feed_dict={x: observaciones, y_: etiquetas}))

0.70625
